
# ResNeXt Grouped Convolution Residual Block (ResNeXt Bloğu) — Temelden Detaya

> Bu notlar; **Basic Residual**, **Pre-activation Residual**, **Bottleneck** ve **ResNeXt (Grouped Conv) Residual Block** yapılarını aynı çerçevede ele alır.  
> Odak: ResNeXt’in ne anlama geldiği, neden ortaya çıktığı ve diğer üç bloktan farkları.

---

## İçindekiler

1. Residual öğrenme fikri (neden residual?)
2. Basic Residual Block (ResNet-18/34)
3. Pre-activation Residual Block (ResNet v2)
4. Bottleneck Residual Block (ResNet-50/101/152)
5. ResNeXt: Grouped Conv Residual Block (Cardinality)
6. Karşılaştırma: Basic vs Pre-Act vs Bottleneck vs ResNeXt
7. Parametre / FLOPs sezgisi (kabaca)
8. PyTorch: Referans implementasyonları
9. Pratik notlar: nerede hangisi seçilir?

---



## 1) Residual Öğrenme: Temel Mantık

Klasik bir konvolüsyonel blokta bir katmanlar dizisi, girdiden doğrudan bir çıktı üretir:

\[
y = H(x)
\]

ResNet fikri şunu söyler: **ağ, doğrudan \(H(x)\)'i öğrenmek yerine** bir "artık" (residual) fonksiyonu öğrensin:

\[
y = x + F(x)
\]

Burada:
- \(x\): skip/identity hattından geçen giriş
- \(F(x)\): main path üzerindeki konvolüsyon + normalizasyon + aktivasyonların ürettiği dönüşüm

### Neden işe yarar?
- Derin ağlarda **gradyan akışını** iyileştirir (skip bağlantı üzerinden).
- Ağ, "gerekirse hiçbir şey yapmama" (identity) davranışına kolayca yakınsar.
- Optimizasyon daha stabil olur: \(F(x)\) küçük düzeltmeler öğrenebilir.

### Boyut uyuşmazlığı
Eğer kanal sayısı / uzamsal boyut (H×W) değişiyorsa, identity hattı da uyarlanır:

\[
y = W_s x + F(x)
\]

Pratikte \(W_s\) çoğu zaman **1×1 Conv (projection)** + BN'dir.

---

> Bu temel denklem, Basic / Pre-Act / Bottleneck / ResNeXt dahil tüm residual blokların iskeletidir. Fark, \(F(x)\)'in nasıl tasarlandığıdır.



## 2) Basic Residual Block (ResNet-18/34)

### Yapı
Main path genellikle şu şekildedir:

- 3×3 Conv → BN → ReLU  
- 3×3 Conv → BN  
- (toplama) + ReLU

\[
F(x) = \text{BN}(\text{Conv}_{3\times3}(\sigma(\text{BN}(\text{Conv}_{3\times3}(x)))))
\]
\[
y = x + F(x) \quad \text{(sonra ReLU)}
\]

### Özellikler
- Basit ve etkilidir.
- Çok derinleşince (50+ katman gibi) hesap maliyeti artar; bu yüzden Bottleneck yaygınlaşır.
- Kanal genişliği (width) arttıkça maliyet hızlı yükselir (3×3 pahalıdır).

---



## 3) Pre-activation Residual Block (ResNet v2)

Pre-activation yaklaşımı, aktivasyon ve normalizasyonu **toplamadan önce** konumlandırır.

### Klasik (v1) vs Pre-Act (v2) farkı
- v1: Conv → BN → ReLU → Conv → BN → (+) → ReLU
- v2: BN → ReLU → Conv → BN → ReLU → Conv → (+)  (sonrasında ekstra ReLU yok ya da mimariye göre değişir)

### Neden ortaya çıktı?
- Skip hattı boyunca "daha temiz" identity akışı sağlar.
- Normalizasyon/aktivasyonlar main path'i şekillendirirken, skip hattı mümkün olduğunca bozulmadan akar.
- Çok derin ağlarda optimizasyonu daha da stabilize eder.

Matematiksel sezgi:
\[
y = x + F(\text{BN}(x))
\]
gibi düşünülebilir (tam form mimariye göre).

### Özet
- Aynı parametre sayısıyla, genellikle daha iyi eğitim dinamiği.
- Modern pratikte bazı ResNet türevleri bu fikri içerir.

---



## 4) Bottleneck Residual Block (ResNet-50/101/152)

Basic blok iki tane 3×3 kullanırken, Bottleneck pahalı 3×3'ü "dar bir kanal alanında" yaptırır.

### Yapı (tipik)
- 1×1 Conv (reduce) → BN → ReLU
- 3×3 Conv (process) → BN → ReLU
- 1×1 Conv (expand) → BN
- (+) → ReLU

Kanallar:
- Giriş: \(C\)
- İç (bottleneck): \(C_b\) (çoğu zaman \(C_b = C/4\))
- Çıkış: \(C_{\text{out}} = \text{expansion} \cdot C_b\) (expansion genellikle 4)

\[
F(x) = \text{Conv}_{1\times1}^{\uparrow}\Big(\text{Conv}_{3\times3}(\text{Conv}_{1\times1}^{\downarrow}(x))\Big)
\]

### Neden avantajlı?
- 3×3 işlemi düşük kanal sayısında yapılır → FLOPs düşer.
- Derinlik artırmak (50/101/152) daha uygulanabilir olur.

---



## 5) ResNeXt: Grouped Convolution Residual Block (Cardinality)

ResNeXt, Bottleneck bloğunun içindeki 3×3 konvolüsyonu **grouped convolution** olarak tasarlar.

### 5.1) Grouped Convolution nedir?
Klasik bir konvolüsyonda \(C_{in}\) kanallı giriş, tüm kanallar birlikte işlenir.  
Grouped conv'ta kanallar **g** gruba ayrılır ve her grup kendi içinde konvolüsyonlanır.

- Group sayısı: \(g\)
- Her gruptaki kanal sayısı: yaklaşık \(C_{in}/g\)

Bu yaklaşım iki şey sağlar:
1. Aynı hesap bütçesinde daha fazla "paralel dönüşüm yolu"
2. Model kapasitesini artırmanın yeni ekseni: **cardinality**

### 5.2) ResNeXt’in fikri: "Split-Transform-Merge"
ResNeXt bloğunu sezgisel olarak şu şekilde düşünebilirsin:

1) **Split:** Özellik kanallarını g gruba böl  
2) **Transform:** Her grup içinde 3×3 conv uygula  
3) **Merge:** Çıktıları birleştir (concatenate/implicit merge)  
4) Skip ile topla: \(y = x + F(x)\)

Aslında grouped conv bunu tek bir op içinde yapar.

### 5.3) ResNeXt bloğunun tipik yapısı (Bottleneck + Grouped Conv)

- 1×1 Conv (reduce)  
- 3×3 **Grouped Conv** (groups = cardinality)  
- 1×1 Conv (expand)  
- (+) → ReLU

\[
F(x)=\text{Conv}_{1\times1}^{\uparrow}\big(\text{GConv}_{3\times3}^{(g)}(\text{Conv}_{1\times1}^{\downarrow}(x))\big)
\]

Buradaki kritik fark: Bottleneck'teki 3×3 yerine **GConv**.

### 5.4) Neden ortaya çıktı?
ResNet’te performansı artırmanın tipik yolları:
- derinliği artır (depth)
- genişliği artır (width)

ResNeXt üçüncü bir yol önerir:
- **cardinality** artır (daha fazla grup → daha fazla paralel dönüşüm yolu)

Bu pratikte genelde şu anlama gelir:
- Aynı FLOPs seviyesinde, group sayısını artırarak daha iyi doğruluk elde edebilme.
- Modeli "çok genişletmeden" kapasiteyi artırma.

### 5.5) Cardinality ne demek?
Cardinality = grouped conv içindeki **group sayısı** (g).  
Örneğin ResNeXt-50 (32x4d) ifadesinde:
- 32: cardinality (group sayısı)
- 4d: her grubun taban genişliği (base width) gibi okunur

---



## 6) Karşılaştırma (Net Farklar)

| Blok | Main path omurgası | Temel amaç | Kritik fark |
|---|---|---|---|
| **Basic** | 3×3 → 3×3 | Basit residual öğrenme | 3×3'ler doğrudan geniş kanalda |
| **Pre-Act** | (BN+ReLU) → Conv ... | Daha stabil eğitim | Aktivasyon/BN toplama öncesine taşınır |
| **Bottleneck** | 1×1 → 3×3 → 1×1 | Hesabı azaltıp derinleşmek | 3×3 dar kanalda |
| **ResNeXt** | 1×1 → **G-3×3** → 1×1 | Aynı bütçede daha iyi kapasite | 3×3 yerine grouped conv + cardinality |

### Kısa sezgi
- **Basic**: “kolay ve sağlam”  
- **Pre-Act**: “aynı iskelet, daha iyi optimizasyon”  
- **Bottleneck**: “3×3’ü ucuzlat”  
- **ResNeXt**: “3×3’ü grup grup paralelleştir, kapasiteyi cardinality ile büyüt”

---



## 7) Parametre / FLOPs Sezgisi (Kabaca)

### 7.1) Standart 3×3 Conv parametre sayısı
\[
\#\text{param} = 3\cdot 3 \cdot C_{in}\cdot C_{out}
\]

### 7.2) Grouped 3×3 Conv parametre sayısı
Group sayısı \(g\) olursa:
- Her grup \(C_{in}/g\) giriş kanalını ve \(C_{out}/g\) çıkış kanalını işler.

\[
\#\text{param} = 3\cdot 3 \cdot \frac{C_{in}}{g}\cdot \frac{C_{out}}{g}\cdot g
= 3\cdot 3 \cdot \frac{C_{in} C_{out}}{g}
\]

Yani parametre/FLOPs kabaca **\(1/g\)** oranında düşer.

### 7.3) Bu ne sağlıyor?
Aynı hesap/parametre bütçesinde:
- ya width (kanal genişliği) bir miktar artırılabilir,
- ya da daha fazla "farklı dönüşüm" elde edilir (cardinality artışı).

---



## 8) PyTorch Referans Implementasyonları

Aşağıdaki kodlar:
- Basic Residual Block (v1)
- Pre-Activation Basic Block (v2)
- Bottleneck Block
- ResNeXt Bottleneck (Grouped Conv)

Notlar:
- `downsample` otomatik olarak 1×1 projection + BN kullanır.
- ResNeXt bloğunda `groups` parametresi cardinality’dir.


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

def conv3x3(in_ch, out_ch, stride=1, groups=1, dilation=1):
    return nn.Conv2d(
        in_ch, out_ch, kernel_size=3, stride=stride,
        padding=dilation, groups=groups, bias=False, dilation=dilation
    )

def conv1x1(in_ch, out_ch, stride=1):
    return nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    r'''
    ResNet-18/34 tarzı Basic Residual Block (v1):
      Conv3x3 -> BN -> ReLU -> Conv3x3 -> BN -> (+skip) -> ReLU
    '''
    expansion = 1

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = conv3x3(in_ch, out_ch, stride=stride)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.conv2 = conv3x3(out_ch, out_ch, stride=1)
        self.bn2   = nn.BatchNorm2d(out_ch)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = F.relu(out, inplace=True)
        return out


class PreActBasicBlock(nn.Module):
    r'''
    ResNet v2 (pre-activation) tarzı Basic Block:
      BN -> ReLU -> Conv3x3 -> BN -> ReLU -> Conv3x3 -> (+skip)
    Not: Toplamadan sonra ekstra ReLU kullanılmayabilir (v2 yaklaşımı).
    '''
    expansion = 1

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_ch)
        self.conv1 = conv3x3(in_ch, out_ch, stride=stride)
        self.bn2 = nn.BatchNorm2d(out_ch)
        self.conv2 = conv3x3(out_ch, out_ch, stride=1)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        out = self.bn1(x)
        out = F.relu(out, inplace=True)

        identity = x
        if self.downsample is not None:
            identity = self.downsample(out)  # pre-activated

        out = self.conv1(out)
        out = self.bn2(out)
        out = F.relu(out, inplace=True)
        out = self.conv2(out)

        out = out + identity
        return out


class Bottleneck(nn.Module):
    r'''
    ResNet-50/101/152 Bottleneck:
      1x1 reduce -> BN -> ReLU
      3x3       -> BN -> ReLU
      1x1 expand -> BN
      (+skip) -> ReLU
    '''
    expansion = 4

    def __init__(self, in_ch, bottleneck_ch, stride=1):
        super().__init__()
        out_ch = bottleneck_ch * self.expansion

        self.conv1 = conv1x1(in_ch, bottleneck_ch)
        self.bn1 = nn.BatchNorm2d(bottleneck_ch)

        self.conv2 = conv3x3(bottleneck_ch, bottleneck_ch, stride=stride)
        self.bn2 = nn.BatchNorm2d(bottleneck_ch)

        self.conv3 = conv1x1(bottleneck_ch, out_ch)
        self.bn3 = nn.BatchNorm2d(out_ch)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)

        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out, inplace=True)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = F.relu(out, inplace=True)
        return out


class ResNeXtBottleneck(nn.Module):
    r'''
    ResNeXt Bottleneck (Grouped Conv):
      1x1 reduce -> BN -> ReLU
      3x3 GROUPED -> BN -> ReLU
      1x1 expand -> BN
      (+skip) -> ReLU

    Parametreler:
      - groups: cardinality
      - base_width: ResNeXt-50 32x4d gibi isimlerdeki '4' kısmı ile ilişkilidir.
    '''
    expansion = 4

    def __init__(self, in_ch, bottleneck_ch, stride=1, groups=32, base_width=4):
        super().__init__()
        
        # Bottleneck_ch = Bu blok asıl kaç kanallık veriyle çalışacak
        # ResNeXt standardında iç genişlik ölçekleme (yaygın form):
        # bottleneck_ch = 64 → “Bu bloğun beyni 64 kanallık”

        width = int(bottleneck_ch * (base_width / 64.0)) * groups

        # ResNeXt diyor ki:
                ### “Ben bu 64 kanalı tek parça işlemeyeceğim.Parçalara böleceğim.”.İşte groups burada giriyor.

        # Groups ::: Elimizdew 64 kanal varsa biz kanal sayısını gropus a bölerek hepsine ayrı ayrı 3x3 uyguluyoruz
                ### Grup 1 → 16 kanal
                ### Grup 2 → 16 kanal
                ### Grup 3 → 16 kanal
                ### rup 4 → 16 kanal
                
        # base_width ne? ::: “Her grubun KAÇ KANALLIK olmasını istiyorum?”
                ### base_width = 4 ::: groups = 32  ========= 32 tane grup olacak.Her grup 4 kanallık olsun.İşte o formülün GERÇEK anlamı
                    ###  width = bottleneck_ch * (base_width / 64) * groups


        out_ch = bottleneck_ch * self.expansion

        self.conv1 = conv1x1(in_ch, width)
        self.bn1 = nn.BatchNorm2d(width)

        self.conv2 = conv3x3(width, width, stride=stride, groups=groups)
        self.bn2 = nn.BatchNorm2d(width)

        self.conv3 = conv1x1(width, out_ch)
        self.bn3 = nn.BatchNorm2d(out_ch)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)

        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out, inplace=True)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = F.relu(out, inplace=True)
        return out


### Hızlı Şekil Kontrolü

Aşağıdaki test, blokların çıktı boyutlarını kontrol eder.


In [2]:
x = torch.randn(2, 64, 56, 56)

b1 = BasicBlock(64, 64, stride=1)
p1 = PreActBasicBlock(64, 64, stride=1)
bn = Bottleneck(64, 64, stride=1)            # out: 256
rx = ResNeXtBottleneck(64, 64, stride=1, groups=32, base_width=4)  # out: 256

with torch.no_grad():
    y_b1 = b1(x)
    y_p1 = p1(x)
    y_bn = bn(x)
    y_rx = rx(x)

y_b1.shape, y_p1.shape, y_bn.shape, y_rx.shape


(torch.Size([2, 64, 56, 56]),
 torch.Size([2, 64, 56, 56]),
 torch.Size([2, 256, 56, 56]),
 torch.Size([2, 256, 56, 56]))


## 9) Pratik Seçim Rehberi

- **BasicBlock**: Küçük/orta derinlikte (18/34) güçlü baseline. Eğitim kolay.
- **Pre-Act**: Çok derin mimarilerde eğitim stabilitesi için tercih edilebilir.
- **Bottleneck**: 50+ katman hedefleniyorsa klasik seçim. Hesap verimliliği iyi.
- **ResNeXt (Grouped Conv)**: Aynı bütçede daha yüksek kapasite istendiğinde iyi aday.  
  - Cardinality (groups) artırımı, bazen width artırmaktan daha verimli olur.
  - Inference tarafında grouped conv kernel verimliliği ve hedef donanım desteği göz önünde bulundurulur.

---

### Tek cümlelik özet
ResNeXt bloğu, residual iskeleti koruyup Bottleneck’in merkezindeki 3×3 işlemi **grouped conv** yaparak kapasiteyi **cardinality** ekseninde ölçekler; bu, onu Basic/Pre-Act/Bottleneck’ten ayıran ana fikirdir.
